In [4]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# get tfidf

In [5]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/IfKnowDebtor/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(1560, 2)


,label,split_text
0,0,她 是 我 二舅
1,0,恩
2,0,我 是 他 的 爸爸
3,0,他 是 我 房东
4,0,我 是 他 的 朋友
5,0,他 是 我 的 同学
6,0,我 是 他 的 前女友
7,0,我 是 他 上司
8,0,我 是 她 同学
9,0,她 是 我 的 邻居


In [6]:
# phrase_vectorizer1 = TfidfVectorizer(ngram_range=(1,3),
#                                     strip_accents='unicode', 
#                                     max_features=100000, 
#                                     analyzer='word',
#                                     sublinear_tf=True,
#                                     token_pattern=r'\w{1,}')
# data.split_text.iloc[:2]
# dd = ['I like apple do you like it', 'I do not like banana']
# phrase_vectorizer1.fit(dd)

# print('transform phrase')
# phrase = phrase_vectorizer1.transform(dd)


In [7]:
data.split_text.iloc[1]

'恩'

In [8]:
# phrase_vectorizer1.vocabulary_

In [9]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<1560x1794 sparse matrix of type '<class 'numpy.float64'>'
	with 17374 stored elements in Compressed Sparse Row format>

# SVC

In [10]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.997435897436


In [11]:
print(clf.predict(phrase))

[0 0 0 ..., 2 2 2]


In [12]:
import pickle
pickle.dump(clf, open("../../savedModel/IDClassifier/l_svc.pickle", "wb"))

In [13]:
aa = pickle.load(open("../../savedModel/IDClassifier/l_svc.pickle", 'rb'))

In [14]:
aa.predict(phrase)

array([0, 0, 0, ..., 2, 2, 2])

# logistic

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.989102564103


In [17]:
print(log_r.predict(phrase))

[0 0 0 ..., 2 2 2]


In [18]:
# l_svc.predict(phrase)-log_r.predict(phrase)

# lightGBM

In [19]:
import lightgbm as lgb

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [21]:
onelabels

array([0, 0, 0, ..., 2, 2, 2])

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [23]:
lables

array([[1, 1, 1]])

In [24]:
data.label.values

array([0, 0, 0, ..., 2, 2, 2])

In [25]:
# lables.todense()

In [26]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.0596154
[10]	valid_0's multi_error: 0.0544872
[15]	valid_0's multi_error: 0.0397436
[20]	valid_0's multi_error: 0.0333333
[25]	valid_0's multi_error: 0.0326923
[30]	valid_0's multi_error: 0.0326923
[35]	valid_0's multi_error: 0.0326923
[40]	valid_0's multi_error: 0.0320513
[45]	valid_0's multi_error: 0.0314103
[50]	valid_0's multi_error: 0.0320513
[55]	valid_0's multi_error: 0.0320513
[60]	valid_0's multi_error: 0.0307692
[65]	valid_0's multi_error: 0.0307692
[70]	valid_0's multi_error: 0.0307692
[75]	valid_0's multi_error: 0.0301282
[80]	valid_0's multi_error: 0.0301282
[85]	valid_0's multi_error: 0.0301282
[90]	valid_0's multi_error: 0.0301282
[95]	valid_0's multi_error: 0.0301282
[100]	valid_0's multi_error: 0.0301282
[105]	valid_0's multi_error: 0.0301282
[110]	valid_0's multi_error: 0.0301282
[115]	valid_0's multi_error: 0.0301282
[120]	valid_0's multi_error: 0.0301282
[125]	valid_0's multi

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [27]:
print(lgbm_model.predict(phrase))

[[  9.98898975e-01   1.64017019e-03   5.88114722e-04]
 [  3.05136455e-01   5.71814898e-02   5.97803469e-01]
 [  9.99506106e-01   6.24331962e-04   5.20170357e-04]
 ..., 
 [  1.49620538e-02   9.19796534e-03   9.93144033e-01]
 [  1.72115937e-02   4.41226232e-03   9.91030890e-01]
 [  2.25423847e-01   2.01652379e-01   5.08035459e-01]]


In [28]:
# lgbm_model.predict(phrase)

In [29]:
phrase

<1560x1794 sparse matrix of type '<class 'numpy.float64'>'
	with 17374 stored elements in Compressed Sparse Row format>

# Save

In [30]:
import pickle

pickle.dump(phrase_vectorizer, open("../../savedModel/IfKnowDebtor/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))


# save linear svc
pickle.dump(clf, open("../../savedModel/IfKnowDebtor/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/IfKnowDebtor/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/IfKnowDebtor/Lgbm.pickle", "wb"))

In [31]:
pickle.load(open("../../savedModel/IfKnowDebtor/Logistic.pickle", 'rb'))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
# test = phrase_vectorizer.transform(['你 说 呢'])
test = phrase_vectorizer.transform(['认 不 到'])


In [33]:
test

<1x1794 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [34]:
clf.predict_proba(test) # linear svc

array([[ 0.01896056,  0.79228   ,  0.18875944]])

In [35]:
log_r.predict_proba(test) # logistic

array([[ 0.3657827 ,  0.25435757,  0.37985973]])

In [36]:
lgbm_model.predict(test) # light gbm

array([[ 0.10449404,  0.75974481,  0.35718896]])

# labeling

In [37]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.792280002391


# save the whole thing

In [38]:
# import jieba
# import numpy as np

# class IfKnowDebtor:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred<threshold):
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from IfKnowDebtor_py import IfKnowDebtor
ifk = IfKnowDebtor(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/IfKnowDebtor/IfKnowDebtor.pickle", "wb"))